## K-armed bandit problem 
# Page 32: A Simple Bandit Algorithm of the book "Introduction to Reinforcement Learning"

In [7]:
import numpy as np
import random
import matplotlib.pyplot as plt
import sys
from math import sqrt, log
%matplotlib inline

In [8]:
k = 10
steps = 1000

In [20]:
def generate_problem(k):
    return np.random.normal(loc=0.0, scale=1, size=k)

In [21]:
def generate_reward(problem, action):
    return np.random.normal(loc=problem[action], scale=1)

In [22]:
def k_bandit(problem, k, steps, exploration_rate):
    Q = {i: 0 for i in range(k)} # 1. Value function
    N = {i: 0 for i in range(k)} # 2. Number of actions, for update rule
    
    for i in range(steps): # 3. Main loop
        explore = random.uniform(0, 1) < exploration_rate 
        if explore:
            action = random.randint(0, k - 1) # 5. Exploration: Choosing random action
        else:
            action = max(Q, key=Q.get) # 6. Choose action with maximum mean reward
            
        reward = generate_reward(problem, action) # 7. Get reward for current action
        N[action] += 1 # 8. Update action number
        Q[action] += (1 / N[action]) * (reward - Q[action]) # 9. Update value dict

## Problem 4 of Assignment 1

Consider a multi-arm bandit problem with $k = 5$ actions, denoted 1, 2, 3, 4, and 5. Consider
applying to this problem a bandit algorithm using \varepsilon-greedy action selection, sample-average
action-value estimates, and initial estimates of $Q_{1}(a)$ = 0 for all $a$. Suppose the initial
sequence of actions and rewards is $A_{1} = 1$, $R_{1} = –2$, $A_{2} = 2$, $R_{2} = 3$, $A_{3} = 3$, $R_{3} = –1$, $A_{4} =2$, $R_4 = 2$, $A_{5} = 3$, $R_{5} = 0$, $A_{6} = 4$, $R_{6} = 5$. On some of these time steps the  case may have
occurred causing an action to be selected at random (assume that when the $\varepsilon$ case occurs,
the arm(s) with the max value is excluded from the random selection).

(a) On which time steps did this definitely occur?

(b) On which time steps could this possibly have occurred?

In [56]:
R = np.array([-2,3,-1,2,0,5])
A = np.array([1,2,3,2,3,4])

actions = 5
iterations = len(A)
table = np.zeros((iterations + 1, actions))

def weird_division(n, d):
    return n / d if d else 0

table[0,:] = np.zeros(actions)
table[1,:] = np.array([1,0,0,0,0])
for t in range(1,iterations+1):
    for a in range(1, actions+1):
        table[t,a-1]= weird_division(np.sum(R[0:t] * np.piecewise(A, [A == a, A != a], [1, 0])[0:t]),np.sum(np.piecewise(A, [A == a, A != a], [1, 0])[0:t]))
        
table

array([[ 0. ,  0. ,  0. ,  0. ,  0. ],
       [-2. ,  0. ,  0. ,  0. ,  0. ],
       [-2. ,  3. ,  0. ,  0. ,  0. ],
       [-2. ,  3. , -1. ,  0. ,  0. ],
       [-2. ,  2.5, -1. ,  0. ,  0. ],
       [-2. ,  2.5, -0.5,  0. ,  0. ],
       [-2. ,  2.5, -0.5,  5. ,  0. ]])

## Exercise 2.2: Bandit Example of the book "Introduction to Reinforcement Learning"

In [278]:
R = np.array([1,1,2,2,0])
A = np.array([1,2,2,2,3])

actions = 4
iterations = len(A)
table = np.zeros((iterations + 1, actions))

def weird_division(n, d):
    return n / d if d else 0

table[0,:] = np.zeros(actions)
table[1,:] = np.array([1,0,0,0])
for t in range(1,iterations+1):
    for a in range(1, actions+1):
        table[t,a-1]= weird_division(np.sum(R[0:t] * np.piecewise(A, [A == a, A != a], [1, 0])[0:t]),np.sum(np.piecewise(A, [A == a, A != a], [1, 0])[0:t]))
        
table

array([[0.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 1.        , 0.        , 0.        ],
       [1.        , 1.5       , 0.        , 0.        ],
       [1.        , 1.66666667, 0.        , 0.        ],
       [1.        , 1.66666667, 0.        , 0.        ]])

## Problem 5 of Assignment 1

This problem will require you to use equation 2.5 from the book to examine the effects of
the step size α on the performance of the estimate. Assume our initial estimate is 0.

- Write a simple python script which computes the most recent update for the following
sequences of rewards using three different values for $\alpha$ ($\alpha = 0.5, 0.25, 1$). Submit the
printed results.

| step 1 2 3 4 5 6 7 8 9 10    	|
|------------------------------	|
| series 1 2 2 2 2 2 2 2 2 2 2 	|
| series 2 2 2 2 2 2 4 4 4 4 4 	|
| series 3 2 4 2 4 2 4 2 4 2 4 	|

- Which step size worked best for which cases? Explain and generalize this for different
types of reward distributions.

In [65]:
import numpy as np

R = np.array([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2],[2, 2, 2, 2, 2, 4, 4, 4, 4, 4],[2, 4, 2, 4, 2, 4, 2, 4, 2, 4]]).T

step_size = 1
steps = 10
series = 3

Q = np.zeros((steps+1, series))

Q[0,:] = np.zeros(series)
for n in range(10):
    Q[n+1,:] = Q[n,:] + step_size * (R[n,:] - Q[n,:])

print(Q)

[[0. 0. 0.]
 [2. 2. 2.]
 [2. 2. 4.]
 [2. 2. 2.]
 [2. 2. 4.]
 [2. 2. 2.]
 [2. 4. 4.]
 [2. 4. 2.]
 [2. 4. 4.]
 [2. 4. 2.]
 [2. 4. 4.]]


## PROBLEM 1 OF ASSIGNMENT 2

Solving linear equations simultaneously

In [3]:
from sympy import *
from sympy.solvers.solveset import linsolve

pi0, pi1, pi2, pi3 = symbols('pi0, pi1, pi2, pi3')
M = Matrix(((-0.75, 0, 0.25, 0.25, 0),(0.25, -0.75, 0.25, 0.25, 0),(0.5, 0.5, -0.75, 0, 0),(0, 0.25, 0.25, -0.5, 0),(1,1,1,1,1)))
system = A, b = M[:, :-1], M[:, -1]
linsolve(system, pi0, pi1, pi2, pi3)

{(0.1875, 0.25, 0.291666666666667, 0.270833333333333)}